In [9]:
import numpy as np

import gzip 
import os
from typing import Tuple, Dict, Type

In [10]:
DATA_HOME = './data'

In [11]:
class LogisticRegression:
    """ A logistic regression model for binary classification.
    """
    
    def __init__(self):
        self._w = None
        self._b = None
    
    def fit(self, X: np.ndarray, y: np.ndarray, l_rate: float = 0.01, beta: float = 0.9, n_epochs: int = 200) -> None:
        """ Initializes the parameters of the model and sets them using 
        batch gradient descent.
        
        :param np.ndarray X: A matrix of observations of shape (n_observations, n_features).
        :param np.ndarray y: A vector of true class labels corresponding to the observations in ''X'',
        of shape (n_observations,).
        :param float l_rate: Learning rate to use for gradient descent.
        :param float beta: Momentum hyperparameter.
        :param int n_epochs: Number of training epochs to run GD for.
        """
        X = X.T
        y = np.expand_dims(y, axis=1).T
        n_features = X.shape[0]
        self._w = np.random.randn(n_features, 1) * np.sqrt(1 / n_features)
        self._b = 0
        self.GD(X, y, l_rate, beta, n_epochs)
    
    def GD(self, X: np.ndarray, y: np.ndarray, l_rate: float, beta: float, n_epochs: int) -> None:
        """ Performs batch gradient descent.

        :param np.ndarray X: A matrix of observations of shape (n_features, n_observations).
        :param np.ndarray y: A matrix of true class labels corresponding to the  observations in ''X'', of shape (1, n_observations).
        :param float l_rate: Learning rate to use for gradient descent.
        :param float beta: Momentum hyperparameter.
        :param int n_epochs: Number of training epochs to run GD for.
        """
        m_dw = np.zeros_like(self._w)
        m_db = 0
        for epoch in range(n_epochs):
            y_hat = self.forward(X)
            dJ_dw, dJ_db = self.backward(X, y, y_hat)
            m_dw = beta * m_dw + (1 - beta) * dJ_dw
            m_db = beta * m_db + (1 - beta) * dJ_db
            self._w = self._w - l_rate * m_dw
            self._b = self._b - l_rate * m_db
    
    def backward(self, X: np.ndarray, y: np.ndarray, y_hat: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """ Computes the derivatives of the cost function (denoted J) wrt. the model's parameters.
        
        :param np.ndarray X: A numpy.ndarray of observations of shape (n_features, n_observations).
        :param np.ndarray y: A matrix of true class labels corresponding to the 
        observations in ''X'', of shape (1, n_observations).
        :param np.ndarray y_hat: A matrix of probabilities of the positive class predicted by the model, of
        shape (1, n_observations).
        :returns np.ndarray dJ_dw: The derivative of the cost function (J) wrt. to the model's weights (''self._w'').
        :returns float dJ_db: The derivative of the cost function (J) wrt. to the model's biases (''self._b'')
        """
        batch_size = X.shape[1]
        dJ_dw = (1 / batch_size) * (X @ (y_hat - y).T)
        dJ_db = (1 / batch_size) * np.sum(y_hat - y)
        
        return dJ_dw, dJ_db
        
    def forward(self, X: np.ndarray) -> np.ndarray:
        """ Computes the output of the model.

        :param np.ndarray X: A matrix of observations of shape (n_features, n_observations).
        :returns np.ndarray: A matrix of probabilities of each observation belonging to the positive
        class, of shape (1, n_observations).
        """
        return LogisticRegression.sigmoid((self._w.T @ X) + self._b)

    def predict(self, X: np.ndarray) -> np.ndarray:
        """ Computes predictions for a given input. Assumes the model is 
        already trained.
        
        :param np.ndarray X: A matrix of observations of shape (n_observations, n_features).
        :returns np.ndarray: A vector of shape (n_observations,), indicating the predicted class for each observation.
        """
        X = X.T
        predicted_probas = self.forward(X).T
        predictions = np.empty_like(predicted_probas)
        predictions[predicted_probas > 0.5] = 1.
        predictions[predicted_probas < 0.5] = 0.
        
        return np.squeeze(predictions)
    
    def random_search(self, X_train: np.ndarray, y_train: np.ndarray, X_val: np.ndarray,
                      y_val: np.ndarray, param_grid: Dict, max_iter: int = 10, verbose: bool = False) -> Dict:
        """ Performs random search for hyperparameter optimization.
        
        :param np.ndarray X_train: A matrix of train observations of shape (n_observations_train, n_features).
        :param np.ndarray y_train: A vector of true class labels corresponding to the observations in ''X_train'',
        of shape (n_observations_train,).
        :param np.ndarray X_val: A matrix of validation observations of shape (n_observations_test, n_features).
        :param np.ndarray y_val: A vector of true class labels corresponding to the observations in ''X_val'',
        of shape (n_observations_test,).
        :param Dict param_grid: A dictionary with hyperparameter names as keys, and tuples of (low, high) indicating
        the range to uniformly sample from as values.
        :param int max_iter: The number hyperparemeter of evaluations to perform.
        :param bool verbose: If true prints the results of every evaluation to stdout.
        :returns Dict best_hyperparameters: A dictionary of hyperparameter-value pairs, with a set of values which 
        resulted in the highest validation set score.
        """
        previous_score = 0
        best_hyperparameters = None
        for i in range(max_iter):
            hyperparameters = {k: np.random.uniform(v[0], v[1]) for k, v in param_grid.items()}
            self.fit(X_train, y_train, **hyperparameters)
            predictions = self.predict(X_val)
            score = LogisticRegression.evaluate(y_val, predictions)
            if verbose:
                print(f'Score: {score}\tHyperparemeters: {hyperparameters}')
            if score > previous_score:
                previous_score = score
                best_hyperparameters = hyperparameters

        return best_hyperparameters 
    
    @staticmethod
    def sigmoid(arr: np.ndarray) -> np.ndarray:
        return 1. / (1. + np.exp(-arr))

    @staticmethod
    def evaluate(y_true: np.ndarray, y_pred: np.ndarray) -> float:
        # calculate accuracy
        return np.sum(y_true == y_pred) / y_true.shape[0]

In [12]:
def read_idx_data(images_fh, labels_fh):
    """ Reads MNIST data in the idx file format from the file handles.
    
    :param images_fh: A file object containing MNIST images.
    :param labels_fh: A file object containing MNIST labels.
    :returns np.ndarray images: A matrix of unrolled images of shape (n_images, n_rows * n_cols).
    :returns np.ndarray labels: A matrix of image labels of shape (n_images,).
    """
    images_fh.seek(4)
    labels_fh.seek(4)
    
    dt_uint32 = np.dtype(np.int32).newbyteorder('B')
    n_images, n_rows, n_cols = np.frombuffer(images_fh.read(12), dtype=dt_uint32)
    n_labels = np.frombuffer(labels_fh.read(4), dtype=dt_uint32)[0]
    assert n_images == n_labels, 'Images and labels do not match.'
    
    dt_uint8 = np.dtype(np.uint8).newbyteorder('B')
    images_buffer = images_fh.read(n_images * n_rows * n_cols)
    images = np.frombuffer(images_buffer, dtype=dt_uint8).astype(np.float64)
    images = images.reshape(n_images, n_rows*n_cols)
    
    labels_buffer = labels_fh.read(n_labels)
    labels = np.frombuffer(labels_buffer, dtype=dt_uint8).astype(np.float64)
    
    return images, labels

def preprocess_data(images, labels):
    """ Excludes MNIST images with labels in (0, 1) and assigns a binary label to the 
    remaining images, where prime numbers are the positive class (1) and the remaining 
    numbers are the negative class (0).
    
    :param np.ndarray images: A matrix of shape (n_images, n_rows * n_cols) containing 
    the MNIST images.
    :param np.ndarray labels: A matrix of shape (n_images,) containing the MNIST labels.
    :returns np.ndarray images: A reduced matrix of images of shape (n_images, n_rows * n_cols).
    :returns np.ndarray labels: An adjusted matrix of image labels of shape (n_images,).
    """
    mask = ~np.isin(labels, (0, 1))
    labels = labels[mask]
    images = images[mask, :]
    labels[np.isin(labels, (2, 3, 5, 7))] = 1
    labels[np.isin(labels, (4, 6, 8, 9))] = 0
    shuffled_index = np.random.permutation(labels.shape[0])
    images = images[shuffled_index, :]
    labels = labels[shuffled_index]
    
    return images, labels

def standardize_data(input_data, m=None, s=None):
    """ Standardizes the input data along the first axis, to zero mean
    and unit variance.
    
    :param np.ndarray input_data: Data to standardize, of shape (n_observations, n_features).
    :param np.ndarray m: The mean to use for standardizing each of the columns, of shape
    (1, n_features).
    :param np.ndarray s: The standard deviation to use for standardizing each of the columns, 
    of shape (1, n_features).
    :returns np.ndarray input_data: Standardized input data, of shape (n_observations, n_features).
    :returns m: Means used for standardizing each of the columns, of shape (1, n_features).
    :returns s: Standard deviations used for standardizing each of the columns, of shape (1, n_features).
    """
    if m is None:
        m = np.mean(input_data, axis=0)
    if s is None:
        s = np.std(input_data, axis=0, ddof=1)
        s[s == 0] = 1
    input_data = (input_data - m) / s
    return input_data, m, s

In [13]:
with gzip.open(os.path.join(DATA_HOME, 'train-images-idx3-ubyte.gz'), 'r') as train_images_fh, \
     gzip.open(os.path.join(DATA_HOME, 'train-labels-idx1-ubyte.gz'), 'r') as train_labels_fh, \
     gzip.open(os.path.join(DATA_HOME, 't10k-images-idx3-ubyte.gz'), 'r') as validation_images_fh, \
     gzip.open(os.path.join(DATA_HOME, 't10k-labels-idx1-ubyte.gz'), 'r') as validation_labels_fh:
    
    train_images, train_labels = read_idx_data(train_images_fh, train_labels_fh)
    validation_images, validation_labels = read_idx_data(validation_images_fh, validation_labels_fh)
    
train_images, train_labels = preprocess_data(train_images, train_labels)
validation_images, validation_labels = preprocess_data(validation_images, validation_labels)
train_images, train_means, train_stds = standardize_data(train_images)
validation_images, _, _ = standardize_data(validation_images, m=train_means, s=train_stds)

In [14]:
lr = LogisticRegression()

In [15]:
param_grid = {'l_rate': (5, 25), 'beta': (0.9, 0.91)}

In [16]:
lr.random_search(train_images, train_labels, validation_images, validation_labels, param_grid, verbose=True)

Score: 0.9156626506024096	Hyperparemeters: {'l_rate': 19.993934282815687, 'beta': 0.9036566642914767}
Score: 0.9222574508560558	Hyperparemeters: {'l_rate': 6.294814438877758, 'beta': 0.9080081054705631}
Score: 0.9223842739378567	Hyperparemeters: {'l_rate': 12.794830859252762, 'beta': 0.9074435926918728}
Score: 0.8843373493975903	Hyperparemeters: {'l_rate': 23.162074809523997, 'beta': 0.9089540526367388}
Score: 0.922003804692454	Hyperparemeters: {'l_rate': 9.334554216616693, 'beta': 0.9050502634341551}
Score: 0.9225110970196576	Hyperparemeters: {'l_rate': 14.940045178638762, 'beta': 0.9071430613239186}
Score: 0.922003804692454	Hyperparemeters: {'l_rate': 5.858018570214243, 'beta': 0.9023724935285407}
Score: 0.9223842739378567	Hyperparemeters: {'l_rate': 12.16275164043645, 'beta': 0.9091266207572261}
Score: 0.9222574508560558	Hyperparemeters: {'l_rate': 13.997005108589311, 'beta': 0.9035348020563608}
Score: 0.9222574508560558	Hyperparemeters: {'l_rate': 18.802007153326038, 'beta': 0.9067

{'l_rate': 14.940045178638762, 'beta': 0.9071430613239186}

In [17]:
param_grid = {'l_rate': (10, 15), 'beta': (0.7, 0.99)}

In [18]:
lr.random_search(train_images, train_labels, validation_images, validation_labels, param_grid, verbose=True)

Score: 0.9109701965757768	Hyperparemeters: {'l_rate': 14.576844242956494, 'beta': 0.8161911894115483}
Score: 0.8961318960050729	Hyperparemeters: {'l_rate': 13.862720697412245, 'beta': 0.763297032968308}
Score: 0.9221306277742549	Hyperparemeters: {'l_rate': 10.422989884187569, 'beta': 0.9411020415748607}
Score: 0.9113506658211794	Hyperparemeters: {'l_rate': 14.293866683574208, 'beta': 0.9879432846543625}
Score: 0.8882688649334178	Hyperparemeters: {'l_rate': 11.567523595612396, 'beta': 0.8046831591624788}
Score: 0.9228915662650602	Hyperparemeters: {'l_rate': 13.23824695611749, 'beta': 0.9636095984547395}
Score: 0.9223842739378567	Hyperparemeters: {'l_rate': 11.367521550301937, 'beta': 0.8549928407320002}
Score: 0.9214965123652504	Hyperparemeters: {'l_rate': 14.209799741359738, 'beta': 0.8698640208779702}
Score: 0.9223842739378567	Hyperparemeters: {'l_rate': 14.52944031479775, 'beta': 0.9281576492622502}
Score: 0.8838300570703869	Hyperparemeters: {'l_rate': 10.56071119199618, 'beta': 0.80

{'l_rate': 13.23824695611749, 'beta': 0.9636095984547395}